In [142]:
!pip install transformers accelerate datasets evaluate -q

In [143]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch import autograd
import numpy as np
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import random
import torch.multiprocessing as mp
import torch.distributed as dist
from torch.nn.parallel import DistributedDataParallel as DDP
import numpy as np
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, GPT2Config,GPT2LMHeadModel,GPT2Tokenizer
from datasets import load_dataset, Dataset

In [210]:
df = pd.read_csv('./TinyStories_evaluated_final.csv')
df1 = pd.read_csv('./TinyStories_unevaluated.csv')

dataset = Dataset.from_pandas(df)
dataset1 = Dataset.from_pandas(df1)

In [211]:
dataset

Dataset({
    features: ['text', 'avg_perplexity'],
    num_rows: 211972
})

In [212]:
dataset1

Dataset({
    features: ['text', 'avg_perplexity'],
    num_rows: 211972
})

In [214]:
dataset['text'][0]

'<|startoftext|>Once there was a little girl named Lizzy. She was 3 years old and loved to take walks. Every morning she would put on her shoes and go outside. One day she decided to go on a special walk to the garden that was far away. She walked for a long time until she heard a noise."Where are you going Lizzy?" asked a voice from behind.Lizzy turned around to see a big man. He was standing on a balance beam and looking at her very carefully."I am going to the garden," said Lizzy.The man smiled. "I can take you there. Stay close and balance on this beam with me."And so Lizzy and the man walked on the balance beam together. Soon enough they arrived at the distant garden. Lizzy smiled and thanked the man. From that day on she always looked forward to her special walks to the garden.<|endoftext|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pa

In [213]:
dataset1['text'][0]

"<|startoftext|>Once upon a time, there was a small bunny called Binky. Binky was very gentle and enjoyed hopping around in the meadow.One day Binky hopped into a strange new place. It was a big room with a big wardrobe. Binky was curious, so he slowly hopped up to the wardrobe and looked inside.The wardrobe opened with a creak and Binky saw an old man sitting inside. The man smiled when he saw Binky and nodded his head. He said, â€œHello there Binky, what are you doing here?â€Binky was a bit scared at first, but the old man was very gentle and spoke kindly. They chatted for a while about the meadow, and about Binky's friends. Then Binky hopped back out of the wardrobe and said goodbye to the old man.The old man smiled and nodded his head. â€œGoodbye Binky and be safe,â€ he said.Binky hopped away and the old man closed the wardrobe and drifted to"

In [202]:
# Replace double spaces with a single space
# formatted_string = ' '.join([''.join(word.split()) for word in dataset['text'][0].split('  ')])

df1['text'] =  df1['text'].map(lambda x: ' '.join([''.join(word.split()) for word in x.split('  ')]))

# Print the formatted string
print(df1['text'][1])

<|startoftext|>Once upon a time, there was a little boy called Jack. Every morning he would go to the park and play with his friends. One day, Jack saw a huge train in the park. He was so excited! He went up to it and began to pick at it. His mum called from across the park: "No, Jack! That's not yours! You can't pick at it!"But Jack ignored her and kept picking at it. His mum called again, so he stopped and looked at her. "It's not nice to pick at things that don't belong to you, Jack," she said. "The train belongs to everyone and it is important that we all look after it for everyone to enjoy."Jack smiled. He now understood the importance of looking after things that belonged to everyone. He made sure he said hello to the train every morning from then on. The moral of the story is to make sure to look after things that belong to everyone, a


In [178]:
df.head(5)

,text,perplexity1,perplexity2,perplexity3,perplexity4,perplexity5
0,"<|startoftext|>Once upon a time, there was a s...",1.788040e+00,0.0,1.330305,1.317098,1.343916
1,"<|startoftext|>Once upon a time, there was a l...",1.828521e+00,0.0,1.266912,1.261080,1.276553
2,"<|startoftext|>Once upon a time, there was a l...",4.082546e+03,0.0,1.256084,1.247739,1.259821
3,"<|startoftext|>Once upon a time, there was a b...",1.384547e+08,0.0,1.369951,1.350689,1.359326
4,<|startoftext|>Anna and Ben are twins. They li...,2.243089e+00,0.0,1.455908,1.439890,1.438228


In [128]:
df1 = dataset.shard(num_shards=10,index=0)
df2 = dataset.shard(num_shards=10,index=1)
df3 = dataset.shard(num_shards=10,index=2)
df4 = dataset.shard(num_shards=10,index=3)
df5 = dataset.shard(num_shards=10,index=4)
df6 = dataset.shard(num_shards=10,index=5)
df7 = dataset.shard(num_shards=10,index=6)
df8 = dataset.shard(num_shards=10,index=7)
df9 = dataset.shard(num_shards=10,index=8)
df10 = dataset.shard(num_shards=10,index=9)

In [129]:
df1[0]

{'text': 'One day, a little girl named Lily found a needle in her room. She knew it was difficult to play with it because it was sharp. Lily wanted to share the needle with her mom, so she could sew a button on her shirt.\n\nLily went to her mom and said, "Mom, I found this needle. Can you share it with me and sew my shirt?" Her mom smiled and said, "Yes, Lily, we can share the needle and fix your shirt."\n\nTogether, they shared the needle and sewed the button on Lily\'s shirt. It was not difficult for them because they were sharing and helping each other. After they finished, Lily thanked her mom for sharing the needle and fixing her shirt. They both felt happy because they had shared and worked together.'}

In [130]:
df2[0]

{'text': 'Once upon a time, there was a little car named Beep. Beep loved to go fast and play in the sun. Beep was a healthy car because he always had good fuel. Good fuel made Beep happy and strong.\n\nOne day, Beep was driving in the park when he saw a big tree. The tree had many leaves that were falling. Beep liked how the leaves fall and wanted to play with them. Beep drove under the tree and watched the leaves fall on him. He laughed and beeped his horn.\n\nBeep played with the falling leaves all day. When it was time to go home, Beep knew he needed more fuel. He went to the fuel place and got more healthy fuel. Now, Beep was ready to go fast and play again the next day. And Beep lived happily ever after.'}

In [131]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2", bos_token='<|startoftext|>', eos_token='<|endoftext|>', pad_token='<|pad|>')
# I'm not really doing anything with the config buheret
configuration = GPT2Config.from_pretrained('gpt2', output_hidden_states=False)

model = GPT2LMHeadModel.from_pretrained("gpt2", config=configuration)
model.resize_token_embeddings(len(tokenizer))
# model1.cuda()

if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})

seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [132]:
tokenizer

Using sep_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.


GPT2Tokenizer(name_or_path='gpt2', vocab_size=50257, model_max_length=1024, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|startoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 'pad_token': '<|pad|>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	50256: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=False),
	50257: AddedToken("<|startoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	50258: AddedToken("<|pad|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [ ]:
# model1 = GPT2LMHeadModel.from_pretrained("", config=configuration)

In [133]:
def preprocess_function(examples):

    return tokenizer([''.join(x) for x in examples["text"]], padding='max_length', truncation=True, max_length=1024, return_tensors="pt")

In [134]:
df1 = df1.map(
    preprocess_function,
    batched=True,
    num_proc=8
)

Map (num_proc=8):   0%|          | 0/21198 [00:00<?, ? examples/s]

In [135]:
df1 = df1.remove_columns(['text'])
df1.set_format("torch")
df1 = df1.shuffle(seed=42)

In [136]:
df1[0]['input_ids']

tensor([ 7454,  2402,   257,  ..., 50258, 50258, 50258])

In [137]:
tokenizer.decode(df1[0]['input_ids'])

"Once upon a time, there was a little girl named Lily. She loved to play outside on her porch. Her porch was big and blue, just like the sky on a sunny day. \n\nOne day, Lily's mom told her they were going to have a surprise visitor. Lily was excited and wondered who it could be. She sat on her blue porch and waited patiently. \n\nSoon, a little bird flew onto the porch. It was a beautiful blue bird with shiny feathers. Lily was amazed and happy to see the bird. The bird chirped and sang a sweet song. Lily knew it was a special moment, and she felt lucky to be there. \n\nFrom that day on, Lily would sit on her blue porch every day, hoping to see the blue bird again. And sometimes, the bird would come back and sing a beautiful song for her. Lily was grateful for the special moments she had on her porch with the blue bird.<|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|>

In [139]:
from transformers import DataCollatorForLanguageModeling
from torch.utils.data import DataLoader
from transformers import get_scheduler

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
train_dataloader = DataLoader(df1, shuffle=True, batch_size=1,collate_fn = data_collator)
optimizer = optim.AdamW(model.parameters(), lr=1e-4)
num_epochs = 2
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

In [194]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    
    for step, batch in enumerate(train_dataloader):

        model.zero_grad()

        outputs = model( input_ids = batch['input_ids'],
                          labels=batch['labels'],
                          token_type_ids=None)

        loss = outputs[0]

        logits = outputs.logits

        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

In [ ]:
model.eval()

In [20]:
tokenizer.decode(df4[0]['input_ids'])

'< | s t a r t o f t e x t | > O   n   c   e       u   p   o   n       a       t   i   m   e,       t   h   e   r   e       w   a   s       a       l   i   t   t   l   e       g   i   r   l       n   a   m   e   d       L   i   l   y.       S   h   e       l   o   v   e   d       t   o       p   l   a   y       o   u   t   s   i   d   e       a   n   d       e   x   p   l   o   r   e.       O   n   e       d   a   y,       s   h   e       f   o   u   n   d       a       l   o   n   g       r   o   p   e       a   n   d       t   h   o   u   g   h   t       i   t       w   a   s       i   n   t   e   r   e   s   t   i   n   g.       S   h   e       d   e   c   i   d   e   d       t   o       p   l   a   y       w   i   t   h       i   t       a   n   d       s   e   e       w   h   a   t       s   h   e       c   o   u   l   d       d   o       w   i   t   h       i   t.       \n   \n   L   i   l   y       s   t   a   r   t   e   d       t   o       s   w   i   n   g       t   h   e    

In [ ]:
rows = {}
rows['perplexity1'] = []
rows['perplexity2'] = []
rows['perplexity3'] = []
rows['perplexity4'] = []
rows['perplexity5'] = []
rows['perplexity6'] = []
rows['perplexity7'] = []
rows['perplexity8'] = []
rows['perplexity9'] = []
rows['perplexity10'] = []
rows['text'] = []

In [22]:
df2 = df2.map(
    preprocess_function,
    batched=True,
    num_proc=4
)
df2 = df2.remove_columns(['text','perplexity1','perplexity2','perplexity3','perplexity4','perplexity5','perplexity6','perplexity7','perplexity8','perplexity9','perplexity10'])
df2.set_format("torch")
df2 = df2.shuffle(seed=42)

df3 = df3.map(
    preprocess_function,
    batched=True,
    num_proc=4
)
df3 = df3.remove_columns(['text','perplexity1','perplexity2','perplexity3','perplexity4','perplexity5','perplexity6','perplexity7','perplexity8','perplexity9','perplexity10])
df3.set_format("torch")
df3 = df3.shuffle(seed=42)

df4 = df4.map(
    preprocess_function,
    batched=True,
    num_proc=4
)
df4 = df4.remove_columns(['text','perplexity1','perplexity2','perplexity3','perplexity4','perplexity5','perplexity6','perplexity7','perplexity8','perplexity9','perplexity10])
df4.set_format("torch")
df4 = df4.shuffle(seed=42)

df5 = df5.map(
    preprocess_function,
    batched=True,
    num_proc=4
)
df5 = df5.remove_columns(['text','perplexity1','perplexity2','perplexity3','perplexity4','perplexity5','perplexity6','perplexity7','perplexity8','perplexity9','perplexity10])
df5.set_format("torch")
df5 = df5.shuffle(seed=42)

Map (num_proc=4):   0%|          | 0/21198 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/21197 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/21198 [00:00<?, ? examples/s]

In [23]:
df6 = df6.map(
    preprocess_function,
    batched=True,
    num_proc=4
)
df6 = df6.remove_columns(['text','perplexity1','perplexity2','perplexity3','perplexity4','perplexity5','perplexity6','perplexity7','perplexity8','perplexity9','perplexity10])
df6.set_format("torch")
df6 = df6.shuffle(seed=42)


df7 = df7.map(
    preprocess_function,
    batched=True,
    num_proc=4
)
df7 = df7.remove_columns(['text','perplexity1','perplexity2','perplexity3','perplexity4','perplexity5','perplexity6','perplexity7','perplexity8','perplexity9','perplexity10])
df7.set_format("torch")
df7 = df7.shuffle(seed=42)

df8 = df8.map(
    preprocess_function,
    batched=True,
    num_proc=4
)
df8 = df8.remove_columns(['text','perplexity1','perplexity2','perplexity3','perplexity4','perplexity5','perplexity6','perplexity7','perplexity8','perplexity9','perplexity10])
df8.set_format("torch")
df8 = df8.shuffle(seed=42)

df9 = df9.map(
    preprocess_function,
    batched=True,
    num_proc=4
)
df9 = df9.remove_columns(['text','perplexity1','perplexity2','perplexity3','perplexity4','perplexity5','perplexity6','perplexity7','perplexity8','perplexity9','perplexity10])
df9.set_format("torch")
df9 = df9.shuffle(seed=42)

df10 = df10.map(
    preprocess_function,
    batched=True,
    num_proc=4
)
df10 = df10.remove_columns(['text','perplexity1','perplexity2','perplexity3','perplexity4','perplexity5','perplexity6','perplexity7','perplexity8','perplexity9','perplexity10])
df10.set_format("torch")
df10 = df10.shuffle(seed=42)

Map (num_proc=4):   0%|          | 0/21197 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/21197 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/21197 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/21197 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/21197 [00:00<?, ? examples/s]

In [24]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

# Iterate through the evaluation dataset
for example in df2:
    inputs = example
    seq_len = inputs['input_ids'].size(0)

    nlls = []
    input_ids = inputs['input_ids'].to(device)
    target_ids = input_ids.clone()

    with torch.no_grad():
        outputs = model1(input_ids, labels=target_ids)

        neg_log_likelihood = outputs.loss

    ppl = torch.exp(neg_log_likelihood)
    rows['perplexity6'].append(ppl)
    rows['text'].append(tokenizer.decode(inputs['input_ids']))
    progress_bar.update(1)
    

  0%|          | 0/14132 [00:00<?, ?it/s]

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [195]:
num_training_steps = 21197

In [26]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

# Iterate through the evaluation dataset
for example in df3:
    inputs = example
    seq_len = inputs['input_ids'].size(0)

    nlls = []
    input_ids = inputs['input_ids'].to(device)
    target_ids = input_ids.clone()

    with torch.no_grad():
        outputs = model1(input_ids, labels=target_ids)

        neg_log_likelihood = outputs.loss

    ppl = torch.exp(neg_log_likelihood)
    rows['perplexity6'].append(ppl)
    rows['text'].append(tokenizer.decode(inputs['input_ids']))
    progress_bar.update(1)

  0%|          | 0/21197 [00:00<?, ?it/s]

In [ ]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))
# Iterate through the evaluation dataset
for example in df4:
    inputs = example
    seq_len = inputs['input_ids'].size(0)

    nlls = []
    input_ids = inputs['input_ids'].to(device)
    target_ids = input_ids.clone()

    with torch.no_grad():
        outputs = model1(input_ids, labels=target_ids)

        neg_log_likelihood = outputs.loss

    ppl = torch.exp(neg_log_likelihood)
    rows['perplexity6'].append(ppl)
    rows['text'].append(tokenizer.decode(inputs['input_ids']))
    progress_bar.update(1)

  0%|          | 0/21197 [00:00<?, ?it/s]

In [ ]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))
# Iterate through the evaluation dataset
for example in df5:
    inputs = example
    seq_len = inputs['input_ids'].size(0)

    nlls = []
    input_ids = inputs['input_ids'].to(device)
    target_ids = input_ids.clone()

    with torch.no_grad():
        outputs = model1(input_ids, labels=target_ids)

        neg_log_likelihood = outputs.loss

    ppl = torch.exp(neg_log_likelihood)
    rows['perplexity6'].append(ppl)
    rows['text'].append(tokenizer.decode(inputs['input_ids']))
    progress_bar.update(1)

In [ ]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))
# Iterate through the evaluation dataset
for example in df6:
    inputs = example
    rows['perplexity6'].append(torch.tensor([0.0],dtype=torch.float32))
    rows['text'].append(tokenizer.decode(inputs['input_ids']))
    progress_bar.update(1)

In [ ]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))
# Iterate through the evaluation dataset
for example in df7:
    inputs = example
    seq_len = inputs['input_ids'].size(0)

    nlls = []
    input_ids = inputs['input_ids'].to(device)
    target_ids = input_ids.clone()

    with torch.no_grad():
        outputs = model1(input_ids, labels=target_ids)

            # loss is calculated using CrossEntropyLoss which averages over valid labels
            # N.B. the model only calculates loss over trg_len - 1 labels, because it internally shifts the labels
            # to the left by 1.
        neg_log_likelihood = outputs.loss

    ppl = torch.exp(neg_log_likelihood)
    rows['perplexity6'].append(ppl)
    rows['text'].append(tokenizer.decode(inputs['input_ids']))
    progress_bar.update(1)

In [ ]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))
# Iterate through the evaluation dataset
for example in df8:
    inputs = example
    seq_len = inputs['input_ids'].size(0)

    nlls = []
    input_ids = inputs['input_ids'].to(device)
    target_ids = input_ids.clone()

    with torch.no_grad():
        outputs = model1(input_ids, labels=target_ids)

            # loss is calculated using CrossEntropyLoss which averages over valid labels
            # N.B. the model only calculates loss over trg_len - 1 labels, because it internally shifts the labels
            # to the left by 1.
        neg_log_likelihood = outputs.loss

    ppl = torch.exp(neg_log_likelihood)
    rows['perplexity6'].append(ppl)
    rows['text'].append(tokenizer.decode(inputs['input_ids']))
    progress_bar.update(1)

In [ ]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))
# Iterate through the evaluation dataset
for example in df9:
    inputs = example
    seq_len = inputs['input_ids'].size(0)

    nlls = []
    input_ids = inputs['input_ids'].to(device)
    target_ids = input_ids.clone()

    with torch.no_grad():
        outputs = model1(input_ids, labels=target_ids)

        neg_log_likelihood = outputs.loss

    ppl = torch.exp(neg_log_likelihood)
    rows['perplexity6'].append(ppl)
    rows['text'].append(tokenizer.decode(inputs['input_ids']))
    progress_bar.update(1)

In [ ]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))
# Iterate through the evaluation dataset
for example in df10:
    inputs = example
    seq_len = inputs['input_ids'].size(0)

    nlls = []
    input_ids = inputs['input_ids'].to(device)
    target_ids = input_ids.clone()

    with torch.no_grad():
        outputs = model1(input_ids, labels=target_ids)

        neg_log_likelihood = outputs.loss

    ppl = torch.exp(neg_log_likelihood)
    rows['perplexity6'].append(ppl)
    rows['text'].append(tokenizer.decode(inputs['input_ids']))
    progress_bar.update(1)

In [ ]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))
# Iterate through the evaluation dataset
for example in df1:
    inputs = example
    seq_len = inputs['input_ids'].size(0)

    nlls = []
    input_ids = inputs['input_ids'].to(device)
    target_ids = input_ids.clone()

    with torch.no_grad():
        outputs = model1(input_ids, labels=target_ids)

        neg_log_likelihood = outputs.loss

    ppl = torch.exp(neg_log_likelihood)
    rows['perplexity6'].append(ppl)
    rows['text'].append(tokenizer.decode(inputs['input_ids']))
    progress_bar.update(1)

In [ ]:
rows['perplexity6'] = [x.item() for x in rows['perplexity6']]

In [207]:
# Append perplexity values to the dataset
dataset_with_perplexity = pd.DataFrame({'text': dataset['text'], 'perplexity1': dataset['perplexity1'],  'perplexity2': dataset['perplexity2'], 'perplexity3': dataset['perplexity3'],'perplexity4':dataset['perplexity4'],'perplexity5':dataset['perplexity5'],'perplexity6':dataset1['perplexity6'],'perplexity7':dataset1['perplexity7'],'perplexity8':dataset1['perplexity8'],'perplexity9':dataset1['perplexity9'],'perplexity10':dataset1['perplexity10']})

In [186]:
dataset_with_perplexity['avg_perplexity'] =  dataset_with_perplexity[['perplexity1', 'perplexity2','perplexity3','perplexity4','perplexity5','perplexity6','perplexity7','perplexity8','perplexity9','perplexity10']].mean(axis=1)

In [208]:
dataset_with_perplexity = dataset_with_perplexity.sort_values('avg_perplexity')
dataset_with_perplexity.drop(['perplexity1', 'perplexity2','perplexity3','perplexity4','perplexity5','perplexity6','perplexity7','perplexity8','perplexity9','perplexity10'], axis=1, inplace=True)
print(dataset_with_perplexity)

                                                     text  avg_perplexity
0       <|startoftext|>Once upon a time, there was a s...    1.126058e+00
1       <|startoftext|>Once upon a time, there was a l...    1.126058e+00
2       <|startoftext|>Once upon a time, there was a l...    1.127786e+00
3       <|startoftext|>Once upon a time, there was a b...    1.134087e+00
4       <|startoftext|>Anna and Ben are twins. They li...    1.167423e+00
...                                                   ...             ...
211967  <|startoftext|>Once upon a time there were two...    3.681808e+18
211968  <|startoftext|>Molly was feeling restless. She...    3.681808e+18
211969  <|startoftext|>Once upon a time there was a li...    3.681808e+18
211970  <|startoftext|>Once upon a time there was a ca...    8.737821e+19
211971  <|startoftext|>John was a kind boy, who enjoye...    1.599574e+20

[211972 rows x 2 columns]


In [209]:
# Save the dataset to a file or use it as needed
dataset_with_perplexity.to_csv("./TinyStories_evaluated_final.csv", index=False)